In [2]:
from pathlib import Path

import random
import math

import numpy as np
from numba import njit, prange

import pandas as pd

from matplotlib import pyplot as plt

In [3]:

def F(x: np.ndarray) -> float:
    return 1/(1+np.exp(-x))


@njit(fastmath=True)
def F_jit(x: np.ndarray) -> float:
    return 1/(1+np.exp(-x))


@njit(fastmath=True)
def dF_jit(x: np.ndarray) -> float:
    y = 1/(1+np.exp(-x))
    return y*(1-y)


def Fh(x: np.ndarray) -> float:
    # return np.clip(x,-1, 1)
    return np.tanh(x)


@njit(fastmath=True)
def Fh_jit(x: np.ndarray) -> np.ndarray:
    # return np.clip(x,-1, 1)
    return np.tanh(x)


@njit(fastmath=True)
def dFh_jit(x: np.ndarray) -> np.ndarray:
    # return np.array([0 if xi <= -1 or xi >= 1 else 1 for xi in x])
    return 1-np.square(np.tanh(x))


@njit
def grads(xBatch: np.ndarray, yBatch: np.ndarray, Wy: np.ndarray, Wh: np.ndarray, Bh:np.ndarray, b:np.ndarray) -> tuple[np.ndarray]:
    dWh = np.zeros(Wh.shape)
    dWy = np.zeros(Wy.shape)
    dBh = np.zeros(Bh.shape)
    db = np.zeros(b.shape)

    Uh = np.zeros((Wh.shape[0], Wh.shape[2]))
    Yh = np.zeros((Wh.shape[0], Wh.shape[1]))
    
    for i in prange(xBatch.shape[0]):

        Uh *= 0
        Yh *= 0

        for j in prange(Wh.shape[0]):
            Uh[j] = xBatch[i] @ Wh[j] + Bh[j]
            Yh[j] = Fh_jit(Uh[j]) @ Wh[j].T

        u = np.sum(Yh, axis=0) @ Wy + b
        y = F_jit(u)
        
        dLdu = 2/Wy.shape[0] * (y-yBatch[i])*dF_jit(u)
        
        for j in prange(Wh.shape[0]):
            dWh[j] += (dLdu @ Wy.T) @ Wh[j] * dFh_jit(Uh[j]) * np.atleast_2d(xBatch[i]).T
            dBh[j] += (dLdu @ Wy.T) @ Wh[j] * dFh_jit(Uh[j])
        
        dWy += dLdu * np.atleast_2d(np.sum(Yh, axis=0)).T
        db += dLdu
    
    return (dWh, dWy, dBh, db)

class RBMPerceptron:
    def __init__(self, nL:int, nH:int, nIn:int, nOut:int) -> None:
        self.nL = nL
        self.nH = nH
        self.nIn = nIn
        self.nOut = nOut
                
        self.Wh: np.ndarray = np.random.uniform(-1, 1, (nL, nIn, nH))
        self.Wy: np.ndarray = np.random.uniform(-1, 1, (nIn, nOut))
        
        self.Bh: np.ndarray = np.zeros((nL, nH))
        self.b: np.ndarray = np.zeros(nOut)


    def predict(self, x:np.ndarray) -> np.ndarray:
        Yh = np.zeros(self.nIn)
        
        for i in range(self.Wh.shape[0]):
            Yh += Fh(x @ self.Wh[i] + self.Bh[i]) @ self.Wh[i].T
            
        return F(Yh @ self.Wy + self.b)


    def train(self, xTrain: np.ndarray, yTrain: np.ndarray, lr, batch_size, max_iter) -> None:
        n = xTrain.shape[0]

        for k in range(max_iter):
            idxs = np.random.choice(a=np.arange(n), size=batch_size, replace=False)
            
            dWh, dWy, dBh, db = grads(xTrain[idxs], yTrain[idxs], self.Wy, self.Wh, self.Bh, self.b)
            
            self.Wh -= lr*dWh
            self.Wy -= lr*dWy
            self.Bh -= lr*dBh
            self.b -= lr*db
        
    
    def loss(self, x: np.ndarray, y: np.ndarray) -> float:
        Yp = np.array([self.predict(xi) for xi in x])
        d = 1/self.nOut * np.sum(np.square(y-Yp), axis=1)
        return 1/y.shape[0] * np.sum(d)

In [4]:
df = pd.read_csv(Path('..', '..', 'data', 'iris_csv.csv'))

for c in df.columns[0:4]:
    df[c] = (df[c]-df[c].mean())/df[c].std()

df['synth1'] = df['petallength']*df['petalwidth']
df['synth2'] = df['sepallength']*df['petallength']
df['synth3'] = df['sepallength']*df['petalwidth']

for name in df['class'].unique():
    df[f'{name}_label'] = df['class'].map(lambda x: 1 if x == name else 0)

In [5]:
test_frac = 0.8

np.random.seed(0)
p = np.random.permutation(df.index.size)

test_size = int(p.size*test_frac)
train_size = int(p.size*(1-test_frac))

idx_test = p[0 : test_size]
idx_train = p[test_size: p.size]

features_columns = ['sepallength', 'sepalwidth', 'petallength', 'petalwidth']
label_columns = ['Iris-setosa_label', 'Iris-versicolor_label', 'Iris-virginica_label']

xTest = np.array(df.iloc[idx_test][features_columns])
yTest = np.array(df.iloc[idx_test][label_columns])

xTrain = np.array(df.iloc[idx_train][features_columns])
yTrain = np.array(df.iloc[idx_train][label_columns])

In [24]:
nL = 3
nH = 4
nIn = 4
nOut = 3

lr = 1e-2
batch_size = 15
max_iter = 10000

model = RBMPerceptron(nL, nH, nIn, nOut)

print('untrained loss: {0:.6f}'.format(model.loss(xTest, yTest)))

model.train(xTrain, yTrain, lr, batch_size, max_iter)

print('trained loss: {0:.6f}'.format(model.loss(xTest, yTest)))

untrained loss: 0.210170
trained loss: 0.030983


In [7]:
for x,y in zip(xTest, yTest):
    print(model.predict(x).round(2), y.round(2))

[0.   0.03 1.  ] [0 0 1]
[0.02 0.98 0.  ] [0 1 0]
[1.   0.03 0.  ] [1 0 0]
[0.   0.12 0.98] [0 0 1]
[1.   0.07 0.  ] [1 0 0]
[0.   0.03 1.  ] [0 0 1]
[1.   0.06 0.  ] [1 0 0]
[0.04 0.94 0.  ] [0 1 0]
[0.02 0.96 0.  ] [0 1 0]
[0.04 0.98 0.  ] [0 1 0]
[0.   0.68 0.29] [0 0 1]
[0.09 0.92 0.  ] [0 1 0]
[0.04 0.97 0.  ] [0 1 0]
[0.02 0.95 0.01] [0 1 0]
[0.03 0.94 0.  ] [0 1 0]
[1.   0.08 0.  ] [1 0 0]
[0.03 0.94 0.01] [0 1 0]
[0.03 0.97 0.  ] [0 1 0]
[1.   0.09 0.  ] [1 0 0]
[1.   0.04 0.  ] [1 0 0]
[0.   0.14 0.97] [0 0 1]
[0.04 0.91 0.01] [0 1 0]
[1.   0.05 0.  ] [1 0 0]
[1.   0.11 0.  ] [1 0 0]
[0.   0.61 0.38] [0 0 1]
[1.   0.05 0.  ] [1 0 0]
[1.   0.03 0.  ] [1 0 0]
[0.05 0.97 0.  ] [0 1 0]
[0.04 0.99 0.  ] [0 1 0]
[1.   0.06 0.  ] [1 0 0]
[0.   0.4  0.63] [0 0 1]
[0.04 0.89 0.01] [0 1 0]
[1.   0.06 0.  ] [1 0 0]
[0.   0.64 0.23] [0 0 1]
[0.   0.04 1.  ] [0 0 1]
[0.03 0.96 0.  ] [0 1 0]
[1.   0.05 0.  ] [1 0 0]
[0.   0.67 0.27] [0 1 0]
[0.05 0.94 0.  ] [0 1 0]
[0.03 0.98 0.  ] [0 1 0]
